In [321]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
bus2402_df = pd.read_csv("./Data/2024년_버스노선별_정류장별_시간대별_승하차_인원_정보(02월).csv")

C:\Users\ITSC\AppData\Local\Temp\ipykernel_844\141047399.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bus2402_df = pd.read_csv("./Data/2024년_버스노선별_정류장별_시간대별_승하차_인원_정보(02월).csv")


In [8]:
bus2402_df.head()

,사용년월,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,...,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,교통수단타입코드,교통수단타입명,등록일자
0,202402,741,741번(진관차고지~헌인릉입구),100000001,1001,종로2가사거리(00075),113,135,87,117,...,231,240,156,209,135,197,111,10,서울간선버스,20240303
1,202402,N37,N37번(송파공영차고지~진관공영차고지),100000001,1001,종로2가사거리(00032),9,17,160,177,...,0,0,0,0,0,0,0,51,서울심야버스,20240303
2,202402,470,470번(상암차고지~안골마을),100000001,1001,종로2가사거리(00066),79,88,0,1,...,238,353,235,305,192,221,106,10,서울간선버스,20240303
3,202402,N37,N37번(진관공영차고지~송파공영차고지),100000001,1001,종로2가사거리(00089),0,0,0,0,...,0,0,0,0,0,0,0,51,서울심야버스,20240303
4,202402,100,100번(하계동~용산구청),100000002,1002,창경궁.서울대학교병원(00031),0,0,0,0,...,18,32,22,54,35,24,3,10,서울간선버스,20240303


In [26]:
busstop_df = pd.read_excel("./Data/서울시버스정류소위치정보(20240305).xlsx")

In [221]:
def convert_to_int(value):
    if isinstance(value, int):
        return value
    else:
        return 0

In [314]:
busstop_df["정류소명"] = busstop_df["정류소명"].map(remove_string)

In [52]:
bus2402_df["역명"].str.slice(stop=-7).unique()

array(['종로2가사거리', '창경궁.서울대학교병원', '명륜3가.성대입구', ..., '김포교통(종점가상)',
       '구미동차고지(기점가상)', '가좌동종점(종점가상)'], dtype=object)

In [ ]:
url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"

request=coordsToaddr

params = {"query" : word}

headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

res = requests.get(url, params = params, headers = headers)

In [21]:
def reverse_geocoding(latitude, longitude):
    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={longitude},{latitude}&output=json"
    headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data.get('status') == 'OK':
            address = data['results'][0]['region']['area1']['name'] + ' ' + data['results'][0]['region']['area2']['name'] + ' ' + data['results'][0]['region']['area3']['name']
            return address
        else:
            print("Reverse geocoding failed.")
    else:
        print("Error:", response.status_code)

# 사용 예시
latitude = 37.5665  # 위도
longitude = 126.9780  # 경도

address = reverse_geocoding(latitude, longitude)
print("주소:", address)


Reverse geocoding failed.
주소: None


In [22]:
def reverse_geocoding(latitude, longitude, api_key_id, api_key):
    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={longitude},{latitude}&output=json"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": api_key_id,
        "X-NCP-APIGW-API-KEY": api_key
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data.get('status') == 'OK':
            address = data['results'][0]['region']['area1']['name'] + ' ' + data['results'][0]['region']['area2']['name'] + ' ' + data['results'][0]['region']['area3']['name']
            return address
        else:
            print("Reverse geocoding failed.")
    else:
        print("Error:", response.status_code)

# 사용 예시
latitude = 37.5665  # 위도
longitude = 126.9780  # 경도
api_key_id = "0tsfmn4cxb"
api_key = "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"

address = reverse_geocoding(latitude, longitude, api_key_id, api_key)
print("주소:", address)


Reverse geocoding failed.
주소: None


In [31]:
curl = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=126.9877522923,37.5698055407&sourcecrs=epsg:4326&output=json&orders=legalcode,admcode"
headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

In [24]:
headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
          "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}

In [32]:
res = requests.get(curl, headers)

In [37]:
res.json()

{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'legalcode',
   'code': {'id': '1111013800', 'type': 'L', 'mappingId': '09110138'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로2가',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.986903,
       'y': 37.57044}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}},
  {'name': 'admcode',
   'code': {'id': '1111061500', 'type': 'A', 'mappingId': '09110615'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {

In [50]:
res.json()["results"][0]["region"]["area2"]["name"]

'종로구'

In [ ]:
{'status': {'code': 0, 'name': 'ok', 'message': 'done'},
 'results': [{'name': 'legalcode',
   'code': {'id': '1111013800', 'type': 'L', 'mappingId': '09110138'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로2가',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.986903,
       'y': 37.57044}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}},
  {'name': 'admcode',
   'code': {'id': '1111061500', 'type': 'A', 'mappingId': '09110615'},
   'region': {'area0': {'name': 'kr',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}},
    'area1': {'name': '서울특별시',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978388,
       'y': 37.56661}},
     'alias': '서울'},
    'area2': {'name': '종로구',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.978835,
       'y': 37.573521}}},
    'area3': {'name': '종로1.2.3.4가동',
     'coords': {'center': {'crs': 'EPSG:4326',
       'x': 126.990262,
       'y': 37.574461}}},
    'area4': {'name': '',
     'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}}]}

In [264]:
bus2402_df.head()

,사용년월,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,...,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,교통수단타입코드,교통수단타입명,등록일자,역명2
0,202402,741,741번(진관차고지~헌인릉입구),100000001,1001,종로2가사거리(00075),113,135,87,117,...,240,156,209,135,197,111,10,서울간선버스,20240303,종로2가사거리
1,202402,N37,N37번(송파공영차고지~진관공영차고지),100000001,1001,종로2가사거리(00032),9,17,160,177,...,0,0,0,0,0,0,51,서울심야버스,20240303,종로2가사거리
2,202402,470,470번(상암차고지~안골마을),100000001,1001,종로2가사거리(00066),79,88,0,1,...,353,235,305,192,221,106,10,서울간선버스,20240303,종로2가사거리
3,202402,N37,N37번(진관공영차고지~송파공영차고지),100000001,1001,종로2가사거리(00089),0,0,0,0,...,0,0,0,0,0,0,51,서울심야버스,20240303,종로2가사거리
4,202402,100,100번(하계동~용산구청),100000002,1002,창경궁.서울대학교병원(00031),0,0,0,0,...,32,22,54,35,24,3,10,서울간선버스,20240303,창경궁.서울대학교병원


In [265]:
busstop_coord_df = bus2402_df.loc[:, ("버스정류장ARS번호", "역명2")]

In [267]:
busstop_coord_df = busstop_coord_df.drop_duplicates()

In [37]:
busstop_coord_df["버스정류장ARS번호"] = busstop_coord_df["버스정류장ARS번호"].map(convert_to_int)

NameError: name 'busstop_coord_df' is not defined

In [281]:
busstop_coord_df["버스정류장ARS번호"].isin(busstop_df["ARS_ID"])

0         True
4         True
23        True
42        True
49        True
         ...  
42341    False
42352    False
42358    False
42364    False
42367    False
Name: 버스정류장ARS번호, Length: 12701, dtype: bool

In [279]:
busstop_coord_df["역명2"].isin(busstop_df["정류소명"]).describe()

count     12701
unique        2
top        True
freq      10543
Name: 역명2, dtype: object

In [283]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")]

,버스정류장ARS번호,역명2,ars_coord,station_coord
4445,4835,옥수역(가상),,
15800,35331,은평공영차고지,,
19553,14148,난지한강공원(야구장),,
32743,0,염곡지하차도북측(가상),,
32918,22574,방배역.서초프라자약국,,
...,...,...,...,...
42340,~,청암자이아파트(가상),,
42352,~,김포교통(기점가상),,
42358,~,김포교통(종점가상),,
42364,~,구미동차고지(기점가상),,


In [327]:
def station_coord(station):
    try:
        a = [busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 4].iloc[0], 
             busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return ""

In [334]:
busstop_df.loc[busstop_df["정류소명"].str.contains("국민은행")].iloc[:, 4].iloc[0]

37.6046629766

In [335]:
busstop_df.loc[busstop_df["정류소명"].str.contains("국민은행")].iloc[:, 3].iloc[0]

126.9608880061

In [332]:
busstop_df["정류소명"].str.contains("무역센터")

0        False
1        False
2        False
3        False
4        False
         ...  
11256    False
11257    False
11258    False
11259    False
11260    False
Name: 정류소명, Length: 11261, dtype: bool

In [343]:
bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\3080729665.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  a = [busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 4].iloc[0],
C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\3080729665.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  busstop_df.loc[busstop_df["정류소명"].str.contains(station)].iloc[:, 3].iloc[0]]
C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\2107388508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["station_coord"] = bus2402_df2["역명3"].map(station_coord)


In [297]:
bus2402_df2["역명3"] = bus2402_df2["역명2"].map(remove_string)

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\651415754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["역명3"] = bus2402_df2["역명2"].map(remove_string)


In [298]:
bus2402_df2.head()

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]","[37.5698055407, 126.9877522923]",종로2가사거리
4,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]","[37.579432932, 126.9965206101]",창경궁.서울대학교병원
23,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]","[37.5825800121, 126.9982508372]",명륜3가.성대입구
42,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]","[37.5685792736, 126.9876130976]",종로2가.삼일교
49,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]","[37.586243, 127.001744]",혜화동로터리.여운형활동터


In [344]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")]

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
19553,14148,난지한강공원(야구장),,,난지한강공원(야구장)
32743,0,염곡지하차도북측(가상),,,염곡지하차도북측
32918,22574,방배역.서초프라자약국,,,방배역서초프라자약국
32948,22878,파이낸셜뉴스.현대렉시온오피스텔,,,파이낸셜뉴스현대렉시온오피스텔
32966,22931,방배열린문화센터(방배4동주민센터),,,방배열린문화센터(방배4동주민센터)
...,...,...,...,...,...
42337,~,구미동차고지(종점가상),,,구미동차고지
42352,~,김포교통(기점가상),,,김포교통
42358,~,김포교통(종점가상),,,김포교통
42364,~,구미동차고지(기점가상),,,구미동차고지


In [338]:
bus2402_df2.loc[(bus2402_df2["ars_coord"] == "") & (bus2402_df2["station_coord"] == "")].iloc[0:20, :]

,버스정류장ARS번호,역명2,ars_coord,station_coord,역명3
19553,14148,난지한강공원(야구장),,,난지한강공원(야구장)
32743,0,염곡지하차도북측(가상),,,염곡지하차도북측
32918,22574,방배역.서초프라자약국,,,방배역.서초프라자약국
32948,22878,파이낸셜뉴스.현대렉시온오피스텔,,,파이낸셜뉴스.현대렉시온오피스텔
32966,22931,방배열린문화센터(방배4동주민센터),,,방배열린문화센터(방배4동주민센터)
33127,22981,방배역3번출구,,,방배역3번출구
33389,23140,임피리얼팰리스호텔.팬트힐루논현,,,임피리얼팰리스호텔.팬트힐루논현
33893,23225,대치SKVIEW아파트.서울교회,,,대치SKVIEW아파트.서울교회
34366,23308,역삼e-편한세상이아파트,,,역삼e-편한세상이아파트
34790,23385,수서119안전센터앞,,,수서119안전센터앞


In [340]:
bus2402_df2["역명3"] = bus2402_df2["역명3"].str.replace(".", "")

C:\Users\ITSC\AppData\Local\Temp\ipykernel_6532\29564721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df2["역명3"] = bus2402_df2["역명3"].str.replace(".", "")


In [342]:
busstop_df["정류소명"] = busstop_df["정류소명"].str.replace(".", "")

In [345]:
busstop_df.head()

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리여운형활동터,127.001744,37.586243,중앙차로


In [350]:
for i in "방배역.서초프라자약국".split("."):
    print(busstop_df[busstop_df["정류소명"].str.contains(i)])

KeyError: 0

In [359]:
busstop_df[busstop_df["정류소명"].str.contains("방배역")].iloc[0, 3]

126.9964723757

In [360]:
busstop_df[busstop_df["정류소명"].str.contains("방배역")].iloc[0, 4]

37.4835892379

In [9]:
bus2402_df["역명2"] = bus2402_df["역명"].str.slice(stop=-7)

In [10]:
bus2402_df3 = bus2402_df.loc[:,("버스정류장ARS번호", "역명2")]

In [15]:
bus2402_df3["역_list"] = bus2402_df3["역명2"].map(remove_string).str.split(".")

In [11]:
bus2402_df3 = bus2402_df3.drop_duplicates()

In [16]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,역_list
0,1001,종로2가사거리,[종로2가사거리]
4,1002,창경궁.서울대학교병원,"[창경궁, 서울대학교병원]"
23,1003,명륜3가.성대입구,"[명륜3가, 성대입구]"
42,1004,종로2가.삼일교,"[종로2가, 삼일교]"
49,1005,혜화동로터리.여운형활동터,"[혜화동로터리, 여운형활동터]"


In [32]:
def ars_coord(ars):
    try:
        a = [busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 4].iloc[0], busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return []

In [31]:
ars_coord(1001)

TypeError: 'Series' object is not callable

In [30]:
busstop_df.loc[busstop_df["ARS_ID"] == 1001].iloc[:, 4].iloc[0]

37.5698055407

In [ ]:
bus2402_df3["버스정류장ARS번호"] = bus2402_df3["버스정류장ARS번호"].map(convert_to_int)

In [43]:
ars_coord_series = bus2402_df3["버스정류장ARS번호"].map(ars_coord)

In [44]:
bus2402_df3["ars_coord"] = ars_coord_series

In [45]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,역_list,ars_coord
0,1001,종로2가사거리,[종로2가사거리],"[37.5698055407, 126.9877522923]"
4,1002,창경궁.서울대학교병원,"[창경궁, 서울대학교병원]","[37.579432932, 126.9965206101]"
23,1003,명륜3가.성대입구,"[명륜3가, 성대입구]","[37.5825800121, 126.9982508372]"
42,1004,종로2가.삼일교,"[종로2가, 삼일교]","[37.5685792736, 126.9876130976]"
49,1005,혜화동로터리.여운형활동터,"[혜화동로터리, 여운형활동터]","[37.586243, 127.001744]"


In [46]:
busstop_df

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
0,100000001,1001,종로2가사거리,126.987752,37.569806,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996521,37.579433,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582580,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로
...,...,...,...,...,...,...
11256,124000334,25995,우성아파트,127.139339,37.550386,일반차로
11257,124000333,25996,우성아파트,127.140046,37.550643,일반차로
11258,124000332,25997,조일약국,127.123596,37.533630,일반차로
11259,124000331,25998,성내시장,127.125497,37.536155,일반차로


In [483]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,ars_coord,역_list
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]",[종로2가사거리]
4,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]","[창경궁, 서울대학교병원]"
23,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]","[명륜3가, 성대입구]"
42,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]","[종로2가, 삼일교]"
49,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]","[혜화동로터리, 여운형활동터]"


In [408]:
for i in ["창경궁", "서울대학교병원"]:
    busstop_df.loc[busstop_df["정류소명"].str.contains(i, regex=False)]

In [413]:
busstop_df.loc[busstop_df["정류소명"].str.contains("창경궁", regex=False)].iloc[0, 4]

37.579432932

In [414]:
busstop_df.loc[busstop_df["정류소명"].str.contains("창경궁", regex=False)].iloc[0, 3]

126.9965206101

In [38]:
def convert_to_int(value):
    if isinstance(value, int):
        return value
    else:
        return 0

In [4]:
def stop_coord(stop):
    return [busstop_df.loc[busstop_df["정류소명"].str.contains(stop, regex=False)].iloc[0, 4], 
            busstop_df.loc[busstop_df["정류소명"].str.contains(stop, regex=False)].iloc[0, 3]]

In [5]:
def stoplist_coord(stoplist):
    a = []
    try:
        for i in stoplist:
            try:
                a = stop_coord(i)

            except:
                pass
    except:
        a = [""]
        
    return a

In [3]:
def ars_coord(ars):
    try:
        a = [busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 4].iloc[0], busstop_df.loc[busstop_df["ARS_ID"] == ars].iloc[:, 3].iloc[0]]
        return a
        
    except:
        return []

In [14]:
def remove_string(text):
    for i in ["(가상)", "(기점가상)", "(종점가상)"]:
        text = text.replace(i, "")
        
    return text

In [48]:
stop_coord = bus2402_df3["역_list"].map(stoplist_coord)

In [49]:
bus2402_df3["stop_coord"] = stop_coord

In [50]:
bus2402_df3.head()

,버스정류장ARS번호,역명2,역_list,ars_coord,stop_coord
0,1001,종로2가사거리,[종로2가사거리],"[37.5698055407, 126.9877522923]","[37.5698055407, 126.9877522923]"
4,1002,창경궁.서울대학교병원,"[창경궁, 서울대학교병원]","[37.579432932, 126.9965206101]","[37.579432932, 126.9965206101]"
23,1003,명륜3가.성대입구,"[명륜3가, 성대입구]","[37.5825800121, 126.9982508372]","[37.5825800121, 126.9982508372]"
42,1004,종로2가.삼일교,"[종로2가, 삼일교]","[37.5685792736, 126.9876130976]","[37.5685792736, 126.9876130976]"
49,1005,혜화동로터리.여운형활동터,"[혜화동로터리, 여운형활동터]","[37.586243, 127.001744]","[37.586243, 127.001744]"


In [168]:
bus2402_df3.loc[(bus2402_df3["ars_coord"].map(lambda x: x == [])) & (bus2402_df3["stop_coord"].map(lambda x: x == []))]

,버스정류장ARS번호,역명2,역_list,ars_coord,stop_coord,coord
38458,0,고등동우체국.성남농협대왕지점,"[고등동우체국, 성남농협대왕지점]",[],[],NaN
38460,0,시흥동농협창고,[시흥동농협창고],[],[],NaN
38461,0,여수대교,[여수대교],[],[],NaN
38462,0,성남우체국,[성남우체국],[],[],NaN
38465,0,단대오거리역.세이브존,"[단대오거리역, 세이브존]",[],[],NaN
...,...,...,...,...,...,...
42337,0,구미동차고지(종점가상),[구미동차고지],[],[],NaN
42352,0,김포교통(기점가상),[김포교통],[],[],NaN
42358,0,김포교통(종점가상),[김포교통],[],[],NaN
42364,0,구미동차고지(기점가상),[구미동차고지],[],[],NaN


In [123]:
busstop_df.loc[busstop_df["정류소명"].str.contains("중앙보훈병원역1번출구")]

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,정류소타입
11041,124000190,25302,중앙보훈병원역1번출구,127.148052,37.527992,일반차로
11042,124000191,25303,중앙보훈병원역1번출구,127.148077,37.527209,일반차로
11148,124900118,25539,9호선중앙보훈병원역1번출구,127.147509,37.527793,마을버스
11223,124900137,25682,9호선중앙보훈병원역1번출구,127.147539,37.527911,마을버스


In [124]:
manual_coord_y = busstop_df.loc[busstop_df["정류소명"].str.contains("중앙보훈병원역1번출구")].iloc[0, 4]
manual_coord_x = busstop_df.loc[busstop_df["정류소명"].str.contains("중앙보훈병원역1번출구")].iloc[0, 3]

In [126]:
bus2402_df3.loc[bus2402_df3.index == 38013].iloc[0, 3].append(manual_coord_y)
bus2402_df3.loc[bus2402_df3.index == 38013].iloc[0, 3].append(manual_coord_x)
bus2402_df3.loc[bus2402_df3.index == 38013]

,버스정류장ARS번호,역명2,역_list,ars_coord,stop_coord
38013,0,중앙보훈병원1번출구,[중앙보훈병원1번출구],"[37.5279919512, 127.1480518756]",[]


In [185]:
bus2402_df3 = bus2402_df3.reset_index(drop = True)

In [173]:
def coord_combine(x):
    if bus2402_df3.loc[x, "ars_coord"] != []:
        a = bus2402_df3.loc[x, "ars_coord"]
        
    else:
        a = bus2402_df3.loc[x, "stop_coord"]
        
    return a

In [152]:
coord_combine(49)

[37.586243, 127.001744]

In [157]:
type(bus2402_df3.index.map(coord_combine))

pandas.core.indexes.base.Index

In [186]:
coords = pd.Series(bus2402_df3.index).map(coord_combine)

In [187]:
coords.loc[coords.map(lambda x: x==[])]

10668    []
10670    []
10671    []
10672    []
10673    []
         ..
12692    []
12697    []
12698    []
12699    []
12700    []
Length: 1645, dtype: object

In [189]:
bus2402_df3["coord"] = pd.Series(bus2402_df3.index).map(coord_combine)

In [192]:
bus2402_df3.iloc[:, [0, 1, 5]].loc[bus2402_df3["coord"].map(lambda x: x != [])]

,버스정류장ARS번호,역명2,coord
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]"
1,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]"
2,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]"
3,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]"
4,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]"
...,...,...,...
12690,0,디지털미디어시티역2번출구(종점가상),"[37.5752078233, 126.8996741287]"
12693,0,우방아파트(종점가상),"[37.6002114009, 127.0094554943]"
12694,0,상계현대2차아파트(종점가상),"[37.6717178437, 127.0509549057]"
12695,0,청암자이아파트(가상),"[37.5345359852, 126.9470270341]"


In [193]:
bus_stop_coord = bus2402_df3.iloc[:, [0, 1, 5]].loc[bus2402_df3["coord"].map(lambda x: x != [])]

In [194]:
bus_stop_coord.head()

,버스정류장ARS번호,역명2,coord
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]"
1,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]"
2,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]"
3,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]"
4,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]"


In [232]:
bus2402_df.columns

Index(['사용년월', '노선번호', '노선명', '표준버스정류장ID', '버스정류장ARS번호', '역명', '00시승차총승객수',
       '00시하차총승객수', '1시승차총승객수', '1시하차총승객수', '2시승차총승객수', '2시하차총승객수', '3시승차총승객수',
       '3시하차총승객수', '4시승차총승객수', '4시하차총승객수', '5시승차총승객수', '5시하차총승객수', '6시승차총승객수',
       '6시하차총승객수', '7시승차총승객수', '7시하차총승객수', '8시승차총승객수', '8시하차총승객수', '9시승차총승객수',
       '9시하차총승객수', '10시승차총승객수', '10시하차총승객수', '11시승차총승객수', '11시하차총승객수',
       '12시승차총승객수', '12시하차총승객수', '13시승차총승객수', '13시하차총승객수', '14시승차총승객수',
       '14시하차총승객수', '15시승차총승객수', '15시하차총승객수', '16시승차총승객수', '16시하차총승객수',
       '17시승차총승객수', '17시하차총승객수', '18시승차총승객수', '18시하차총승객수', '19시승차총승객수',
       '19시하차총승객수', '20시승차총승객수', '20시하차총승객수', '21시승차총승객수', '21시하차총승객수',
       '22시승차총승객수', '22시하차총승객수', '23시승차총승객수', '23시하차총승객수', '교통수단타입코드',
       '교통수단타입명', '등록일자', '역명2'],
      dtype='object')

In [234]:
bus2402_df.columns[4]

'버스정류장ARS번호'

In [236]:
bus2402_df.columns[[-1, 4, 0]]

Index(['역명2', '버스정류장ARS번호', '사용년월'], dtype='object')

In [222]:
bus_in = bus2402_df.iloc[:, 6:54:2].sum(axis = 1)
bus_out = bus2402_df.iloc[:, 7:55:2].sum(axis = 1)

In [223]:
bus_out = bus2402_df.iloc[:, 7:55:2].sum(axis = 1)

In [242]:
bus2402_df4 = bus2402_df.iloc[:, [ -1, 4, 0]]

In [247]:
bus2402_df4.head()

,역명2,버스정류장ARS번호,사용년월,탑승인원,하차인원
0,종로2가사거리,1001,202402,5640,6362
1,종로2가사거리,1001,202402,305,281
2,종로2가사거리,1001,202402,7215,5968
3,종로2가사거리,1001,202402,428,220
4,창경궁.서울대학교병원,1002,202402,2717,3547


In [271]:
bus_stop_coord.groupby(by = "역명2").count()

,버스정류장ARS번호,coord
역명2,,
(구)경남호텔앞,1,1
(구)법화사,2,2
11단지,1,1
123전자타운.2001아울렛,2,2
12번지건영아파트,1,1
...,...,...
힐스테이트신촌.신촌푸르지오1단지후문,1,1
힐스테이트에코송파,1,1
힐스테이트입구,1,1


In [244]:
bus2402_df4["탑승인원"] = bus_in

C:\Users\ITSC\AppData\Local\Temp\ipykernel_844\2219773292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df4["탑승인원"] = bus_in


In [246]:
bus2402_df4["하차인원"] = bus_out

C:\Users\ITSC\AppData\Local\Temp\ipykernel_844\362177271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus2402_df4["하차인원"] = bus_out


In [332]:
bus2402_df5 = pd.merge(bus2402_df4, bus_stop_coord, on = ("역명2", "버스정류장ARS번호"), how = "left")

In [333]:
bus2402_df5.head()

,역명2,버스정류장ARS번호,사용년월,탑승인원,하차인원,coord
0,종로2가사거리,1001,202402,5640,6362,"[37.5698055407, 126.9877522923]"
1,종로2가사거리,1001,202402,305,281,"[37.5698055407, 126.9877522923]"
2,종로2가사거리,1001,202402,7215,5968,"[37.5698055407, 126.9877522923]"
3,종로2가사거리,1001,202402,428,220,"[37.5698055407, 126.9877522923]"
4,창경궁.서울대학교병원,1002,202402,2717,3547,"[37.579432932, 126.9965206101]"


In [304]:
def coord_gu(coord):
    curl = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={coord[0]},{coord[1]}&sourcecrs=epsg:4326&output=json&orders=legalcode,admcode"
    headers = {"X-NCP-APIGW-API-KEY-ID" : "0tsfmn4cxb",
              "X-NCP-APIGW-API-KEY" : "WNsIKG0477VBDWIIqWiS51oF0Y59MNWOSZe0jntZ"}
    
    res = requests.get(curl, headers)
    
    return res

In [301]:
a = coord_gu([37.5698055407, 126.9877522923])

In [302]:
a.json()["results"][0]["region"]["area2"]["name"]

'종로구'

In [299]:
a

<Response [200]>

In [309]:
busstop_df["coord_xy"] = list(busstop_df[])

,X좌표,Y좌표
0,126.987752,37.569806
1,126.996521,37.579433
2,126.998251,37.582580
3,126.987613,37.568579
4,127.001744,37.586243
...,...,...
11256,127.139339,37.550386
11257,127.140046,37.550643
11258,127.123596,37.533630
11259,127.125497,37.536155


In [312]:
busstop_df

RangeIndex(start=0, stop=11261, step=1)

In [316]:
busstop_df.iloc[:, 3], busstop_df.iloc[:, 4]

(0        126.987752
 1        126.996521
 2        126.998251
 3        126.987613
 4        127.001744
             ...    
 11256    127.139339
 11257    127.140046
 11258    127.123596
 11259    127.125497
 11260    127.127337
 Name: X좌표, Length: 11261, dtype: float64,
 0        37.569806
 1        37.579433
 2        37.582580
 3        37.568579
 4        37.586243
            ...    
 11256    37.550386
 11257    37.550643
 11258    37.533630
 11259    37.536155
 11260    37.540343
 Name: Y좌표, Length: 11261, dtype: float64)

In [317]:
# 두 개의 열을 결합하여 리스트로 변환하는 함수 정의
def combine_columns(row):
    return [row['X좌표'], row['Y좌표']]

# apply 메서드를 사용하여 새로운 열 생성
busstop_df['combined'] = busstop_df.apply(combine_columns, axis=1)

In [322]:
tqdm.pandas()

In [323]:
busstop_df["address"] = busstop_df["combined"].progress_map(coord_gu)

100%|██████████| 11261/11261 [45:38<00:00,  4.11it/s] 


In [328]:
busstop_df["address"][0].json()["results"][0]["region"]["area2"]["name"]

'종로구'

In [330]:
busstop_df["address"] = busstop_df["address"].map(lambda x: x.json()["results"][0]["region"]["area2"]["name"])

In [400]:
busstop_df.to_csv("./bus_stop_addres.csv")

In [352]:
bus_stop_coord.head()

,버스정류장ARS번호,역명2,coord,coordx,coordy
0,1001,종로2가사거리,"[37.5698055407, 126.9877522923]",126.987752,37.569806
1,1002,창경궁.서울대학교병원,"[37.579432932, 126.9965206101]",126.996521,37.579433
2,1003,명륜3가.성대입구,"[37.5825800121, 126.9982508372]",126.998251,37.582580
3,1004,종로2가.삼일교,"[37.5685792736, 126.9876130976]",126.987613,37.568579
4,1005,혜화동로터리.여운형활동터,"[37.586243, 127.001744]",127.001744,37.586243


In [350]:
bus_stop_coord["coordx"] = bus_stop_coord["coord"].map(lambda x: x[1])

In [351]:
bus_stop_coord["coordy"] = bus_stop_coord["coord"].map(lambda x: x[0])

In [365]:
bus_stop_address = pd.merge(bus_stop_coord, busstop_df, how = "left", left_on = ("coordx", "coordy"), right_on = ("X좌표", "Y좌표")).iloc[:, [1, 0, -2, -1]]

In [389]:
bus_stop_address.groupby("address").count()

,역명2,버스정류장ARS번호,combined
address,,,
강남구,474,474,474
강동구,318,318,318
강북구,432,432,432
강서구,577,577,577
관악구,476,476,476
광명시,4,4,4
광진구,296,296,296
구로구,503,503,503
금천구,354,354,354


In [373]:
bus2402_df5

,역명2,버스정류장ARS번호,사용년월,탑승인원,하차인원,coord
0,종로2가사거리,1001,202402,5640,6362,"[37.5698055407, 126.9877522923]"
1,종로2가사거리,1001,202402,305,281,"[37.5698055407, 126.9877522923]"
2,종로2가사거리,1001,202402,7215,5968,"[37.5698055407, 126.9877522923]"
3,종로2가사거리,1001,202402,428,220,"[37.5698055407, 126.9877522923]"
4,창경궁.서울대학교병원,1002,202402,2717,3547,"[37.579432932, 126.9965206101]"
...,...,...,...,...,...,...
42365,구미동차고지(종점가상),~,202402,0,51,NaN
42366,구미동차고지(종점가상),~,202402,0,3,NaN
42367,가좌동종점(종점가상),~,202402,0,69,NaN
42368,가좌동종점(종점가상),~,202402,0,19,NaN


In [394]:
bus2402_df6 = pd.merge(bus2402_df5, bus_stop_address, on = ("역명2", "버스정류장ARS번호"), how = "left").dropna()

In [397]:
bus2402_df6.loc[bus2402_df6["address"] == "강남구"]

,역명2,버스정류장ARS번호,사용년월,탑승인원,하차인원,coord,combined,address
31243,뱅뱅사거리,22006,202402,3196,3220,"[37.486653, 127.033097]","[127.033097, 37.486653]",강남구
31244,뱅뱅사거리,22006,202402,2520,5437,"[37.486653, 127.033097]","[127.033097, 37.486653]",강남구
31245,뱅뱅사거리,22006,202402,7695,1157,"[37.486653, 127.033097]","[127.033097, 37.486653]",강남구
31246,뱅뱅사거리,22006,202402,3170,3173,"[37.486653, 127.033097]","[127.033097, 37.486653]",강남구
31247,뱅뱅사거리,22006,202402,4625,2073,"[37.486653, 127.033097]","[127.033097, 37.486653]",강남구
...,...,...,...,...,...,...,...,...
31486,신사역,22016,202402,6016,3771,"[37.512972, 127.020607]","[127.020607, 37.512972]",강남구
31487,신사역,22016,202402,582,425,"[37.512972, 127.020607]","[127.020607, 37.512972]",강남구
31488,신사역,22016,202402,616,495,"[37.512972, 127.020607]","[127.020607, 37.512972]",강남구
31489,신사역,22016,202402,10107,4800,"[37.512972, 127.020607]","[127.020607, 37.512972]",강남구


In [386]:
bus2402_df6.loc[:, ("하차인원", "address")].groupby(by = "address").sum()

,하차인원
address,
강남구,501617
강북구,4287818
강서구,4249595
관악구,6626777
광명시,9653
광진구,2659974
구로구,4455497
금천구,3290841
노원구,3858257


In [399]:
bus2402_df6.groupby("address").count()

,역명2,버스정류장ARS번호,사용년월,탑승인원,하차인원,coord,combined
address,,,,,,,
강남구,135,135,135,135,135,135,135
강북구,1244,1244,1244,1244,1244,1244,1244
강서구,2090,2090,2090,2090,2090,2090,2090
관악구,1699,1699,1699,1699,1699,1699,1699
광명시,10,10,10,10,10,10,10
광진구,899,899,899,899,899,899,899
구로구,1497,1497,1497,1497,1497,1497,1497
금천구,991,991,991,991,991,991,991
노원구,1665,1665,1665,1665,1665,1665,1665
